##### Copyright 2025 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# cp_sat_example

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/sat/cp_sat_example.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/sat/samples/cp_sat_example.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [ ]:
%pip install ortools


Simple solve.

In [ ]:
from ortools.linear_solver import pywraplp
import math

# =============================================================================
#  PARAMETERS globales (GLOBAL PARAMETERS)
# =============================================================================
# Các hằng số này giúp dễ dàng điều chỉnh "luật chơi" của bài toán
# mà không cần sửa logic bên trong các hàm.

""" Test Data lớn """
# Số học viên tối thiểu để mở một lớp
# MIN_STUDENT_PER_CLASS = 20

# # Trọng số cơ bản khi xếp được một sinh viên vào lớp
# STUDENT_BASE_WEIGHT = 1000

# # Điểm cộng cho mỗi buổi học sinh viên đã lỡ (khuyến khích xếp lớp cho SV này)
# MISSED_SESSION_WEIGHT = 5

# # Điểm cộng dựa trên số session đã đăng ký (ưu tiên SV đăng ký ít)
# # Dùng 1/số session để SV nào đăng ký càng ít thì điểm càng cao
# REGISTERED_SESSIONS_WEIGHT = 20

# # Điểm phạt khi mở một lớp mới
# CLASS_OPEN_PENALTY = -9500

# # Hệ số phạt cho việc sử dụng phòng lớn (khuyến khích dùng phòng vừa đủ)
# # Giá trị càng lớn, solver càng "ngại" dùng phòng to.
# # Nên nhỏ hơn nhiều so với STUDENT_BASE_WEIGHT.
# CAPACITY_PENALTY_FACTOR = 1
# sessions = {
#     # HUST - Tech
#     "S01PROG": {"subject": "Advanced Programming", "facility": "HUST", "slots": [{"begin": 8, "end": 11}], "registers": ['SV001', 'SV002', 'SV003', 'SV004', 'SV005', 'SV006', 'SV007', 'SV008', 'SV009', 'SV010', 'SV011', 'SV012', 'SV013', 'SV014', 'SV015', 'SV016', 'SV017', 'SV018', 'SV019', 'SV020', 'SV021', 'SV022', 'SV023', 'SV024', 'SV025', 'SV026', 'SV027', 'SV028', 'SV029', 'SV030', 'SV031', 'SV032', 'SV033', 'SV034', 'SV035', 'SV036', 'SV037']},
#     "S02DSA": {"subject": "DSA", "facility": "HUST", "slots": [{"begin": 13, "end": 16}], "registers": ['SV001', 'SV002', 'SV003', 'SV004', 'SV005', 'SV006', 'SV007', 'SV008', 'SV009', 'SV010', 'SV011', 'SV012', 'SV013', 'SV014', 'SV015', 'SV038', 'SV039', 'SV040', 'SV041', 'SV042', 'SV043', 'SV044', 'SV045', 'SV046', 'SV047', 'SV048', 'SV049', 'SV050', 'SV051', 'SV052', 'SV053', 'SV054', 'SV055', 'SV056']},
#     "S03ML": {"subject": "Machine Learning", "facility": "HUST", "slots": [{"begin": 25, "end": 28}], "registers": ['SV016', 'SV017', 'SV018', 'SV019', 'SV020', 'SV021', 'SV022', 'SV023', 'SV024', 'SV025', 'SV057', 'SV058', 'SV059', 'SV060', 'SV061', 'SV062', 'SV063', 'SV064', 'SV065', 'SV066', 'SV067', 'SV068', 'SV069', 'SV070', 'SV071', 'SV072', 'SV073', 'SV074', 'SV075']},
#     "S04CN": {"subject": "Computer Networks", "facility": "HUST", "slots": [{"begin": 28, "end": 31}], "registers": ['SV026', 'SV027', 'SV028', 'SV029', 'SV030', 'SV031', 'SV032', 'SV033', 'SV034', 'SV035', 'SV036', 'SV037', 'SV076', 'SV077', 'SV078', 'SV079', 'SV080', 'SV081', 'SV082', 'SV083', 'SV084', 'SV085', 'SV086', 'SV087', 'SV088', 'SV089', 'SV090', 'SV091', 'SV092', 'SV093']},
#     "S05SE": {"subject": "Software Engineering", "facility": "HUST", "slots": [{"begin": 33, "end": 36}], "registers": ['SV038', 'SV039', 'SV040', 'SV041', 'SV042', 'SV043', 'SV044', 'SV045', 'SV046', 'SV047', 'SV048', 'SV049', 'SV050', 'SV094', 'SV095', 'SV096', 'SV097', 'SV098', 'SV099', 'SV100', 'SV101', 'SV102', 'SV103', 'SV104', 'SV105', 'SV106', 'SV107', 'SV108', 'SV109', 'SV110']},

#     # VNU - Economics & Social Sciences
#     "S06MICRO": {"subject": "Microeconomics", "facility": "VNU", "slots": [{"begin": 50, "end": 53}], "registers": ['SV051', 'SV052', 'SV053', 'SV054', 'SV055', 'SV056', 'SV057', 'SV058', 'SV059', 'SV060', 'SV111', 'SV112', 'SV113', 'SV114', 'SV115', 'SV116', 'SV117', 'SV118', 'SV119', 'SV120']},
#     "S07MACRO": {"subject": "Macroeconomics", "facility": "VNU", "slots": [{"begin": 53, "end": 56}], "registers": ['SV061', 'SV062', 'SV063', 'SV064', 'SV065', 'SV066', 'SV067', 'SV068', 'SV069', 'SV070', 'SV121', 'SV122', 'SV123', 'SV124', 'SV125', 'SV126', 'SV127', 'SV128', 'SV129', 'SV130']},
#     "S08SOC": {"subject": "Sociology", "facility": "VNU", "slots": [{"begin": 58, "end": 61}], "registers": ['SV071', 'SV072', 'SV073', 'SV074', 'SV075', 'SV076', 'SV077', 'SV078', 'SV079', 'SV080', 'SV131', 'SV132', 'SV133', 'SV134', 'SV135', 'SV136', 'SV137', 'SV138', 'SV139', 'SV140']},
#     "S09PSY": {"subject": "Psychology", "facility": "VNU", "slots": [{"begin": 61, "end": 64}], "registers": ['SV081', 'SV082', 'SV083', 'SV084', 'SV085', 'SV086', 'SV087', 'SV088', 'SV089', 'SV090', 'SV141', 'SV142', 'SV143', 'SV144', 'SV145', 'SV146', 'SV147', 'SV148', 'SV149', 'SV150']},
#     "S10PM": {"subject": "Project Management", "facility": "VNU", "slots": [{"begin": 65, "end": 68}], "registers": ['SV001', 'SV010', 'SV020', 'SV030', 'SV040', 'SV050', 'SV060', 'SV070', 'SV080', 'SV090', 'SV100', 'SV110', 'SV120', 'SV130', 'SV140', 'SV150']},

#     # HANU - Languages & Culture
#     "S11ENGC1": {"subject": "English C1", "facility": "HANU", "slots": [{"begin": 72, "end": 75}], "registers": ['SV091', 'SV092', 'SV093', 'SV094', 'SV095', 'SV096', 'SV097', 'SV098', 'SV099', 'SV100', 'SV101', 'SV102', 'SV103', 'SV104', 'SV105', 'SV106', 'SV107', 'SV108', 'SV109', 'SV110']},
#     "S12JAPN3": {"subject": "Japanese N3", "facility": "HANU", "slots": [{"begin": 75, "end": 78}], "registers": ['SV111', 'SV112', 'SV113', 'SV114', 'SV115', 'SV116', 'SV117', 'SV118', 'SV119', 'SV120', 'SV121', 'SV122', 'SV123', 'SV124', 'SV125']},
#     "S13KORT2": {"subject": "Korean Topik 2", "facility": "HANU", "slots": [{"begin": 80, "end": 83}], "registers": ['SV126', 'SV127', 'SV128', 'SV129', 'SV130', 'SV131', 'SV132', 'SV133', 'SV134', 'SV135', 'SV136', 'SV137', 'SV138', 'SV139', 'SV140']},
#     "S14LIT": {"subject": "Literature Analysis", "facility": "HANU", "slots": [{"begin": 83, "end": 86}], "registers": ['SV141', 'SV142', 'SV143', 'SV144', 'SV145', 'SV146', 'SV147', 'SV148', 'SV149', 'SV150', 'SV005', 'SV015', 'SV025', 'SV035', 'SV045']},
#     "S15IRS": {"subject": "Intl Relations", "facility": "HANU", "slots": [{"begin": 86, "end": 89}], "registers": ['SV055', 'SV065', 'SV075', 'SV085', 'SV095', 'SV105', 'SV115', 'SV125', 'SV135', 'SV145']},

#     # NEU - Business & Finance
#     "S16ACC": {"subject": "Accounting", "facility": "NEU", "slots": [{"begin": 96, "end": 99}], "registers": ['SV001', 'SV002', 'SV003', 'SV004', 'SV005', 'SV006', 'SV007', 'SV008', 'SV009', 'SV010', 'SV111', 'SV112', 'SV113', 'SV114', 'SV115']},
#     "S17AUD": {"subject": "Auditing", "facility": "NEU", "slots": [{"begin": 99, "end": 102}], "registers": ['SV011', 'SV012', 'SV013', 'SV014', 'SV015', 'SV016', 'SV017', 'SV018', 'SV019', 'SV020', 'SV116', 'SV117', 'SV118', 'SV119', 'SV120']},
#     "S18HRM": {"subject": "Human Resources", "facility": "NEU", "slots": [{"begin": 104, "end": 107}], "registers": ['SV021', 'SV022', 'SV023', 'SV024', 'SV025', 'SV026', 'SV027', 'SV028', 'SV029', 'SV030', 'SV121', 'SV122', 'SV123', 'SV124', 'SV125']},
#     "S19INV": {"subject": "Investment", "facility": "NEU", "slots": [{"begin": 107, "end": 110}], "registers": ['SV031', 'SV032', 'SV033', 'SV034', 'SV035', 'SV036', 'SV037', 'SV038', 'SV039', 'SV040', 'SV126', 'SV127', 'SV128', 'SV129', 'SV130']},
#     "S20BANA": {"subject": "Business Analytics", "facility": "NEU", "slots": [{"begin": 112, "end": 115}], "registers": ['SV041', 'SV042', 'SV043', 'SV044', 'SV045', 'SV046', 'SV047', 'SV048', 'SV049', 'SV050', 'SV131', 'SV132', 'SV133', 'SV134', 'SV135']}
# }

# rooms = {
#     # HUST Rooms
#     "HUST_D8_101": {"busy": [{"begin": 12, "end": 13}], "facility": "HUST", "capacity": 30},
#     "HUST_D8_202": {"busy": [{"begin": 29, "end": 30}], "facility": "HUST", "capacity": 40},
#     "HUST_TC_301": {"busy": [], "facility": "HUST", "capacity": 50},
#     "HUST_TC_405": {"busy": [{"begin": 17, "end": 19}], "facility": "HUST", "capacity": 60},
#     "HUST_D6_501": {"busy": [{"begin": 35, "end": 38}], "facility": "HUST", "capacity": 80},
#     "HUST_D9_303": {"busy": [{"begin": 8, "end": 10}], "facility": "HUST", "capacity": 100},
#     "HUST_D5_101": {"busy": [{"begin": 15, "end": 17}], "facility": "HUST", "capacity": 120},

#     # VNU Rooms
#     "VNU_E1_101": {"busy": [{"begin": 56, "end": 57}], "facility": "VNU", "capacity": 30},
#     "VNU_E2_202": {"busy": [], "facility": "VNU", "capacity": 40},
#     "VNU_G2_303": {"busy": [{"begin": 12, "end": 13}, {"begin": 56, "end": 57}], "facility": "VNU", "capacity": 50},
#     "VNU_H1_404": {"busy": [{"begin": 62, "end": 65}], "facility": "VNU", "capacity": 60},
#     "VNU_H2_505": {"busy": [{"begin": 50, "end": 52}], "facility": "VNU", "capacity": 80},
#     "VNU_I1_201": {"busy": [{"begin": 68, "end": 70}], "facility": "VNU", "capacity": 100},
#     "VNU_I2_302": {"busy": [], "facility": "VNU", "capacity": 120},

#     # HANU Rooms
#     "HANU_A1_101": {"busy": [{"begin": 78, "end": 79}], "facility": "HANU", "capacity": 30},
#     "HANU_A2_202": {"busy": [{"begin": 102, "end": 103}], "facility": "HANU", "capacity": 40},
#     "HANU_B2_303": {"busy": [], "facility": "HANU", "capacity": 50},
#     "HANU_C5_404": {"busy": [{"begin": 83, "end": 85}], "facility": "HANU", "capacity": 60},
#     "HANU_D3_505": {"busy": [{"begin": 73, "end": 74}], "facility": "HANU", "capacity": 80},

#     # NEU Rooms
#     "NEU_A1_101": {"busy": [], "facility": "NEU", "capacity": 30},
#     "NEU_A1_202": {"busy": [{"begin": 102, "end": 104}], "facility": "NEU", "capacity": 40},
#     "NEU_B2_303": {"busy": [{"begin": 98, "end": 99}], "facility": "NEU", "capacity": 50},
#     "NEU_C1_404": {"busy": [], "facility": "NEU", "capacity": 60},
#     "NEU_D2_505": {"busy": [{"begin": 110, "end": 112}], "facility": "NEU", "capacity": 80},
#     "NEU_E1_201": {"busy": [{"begin": 96, "end": 98}], "facility": "NEU", "capacity": 100},
#     "NEU_F2_302": {"busy": [], "facility": "NEU", "capacity": 120}
# }

# students = {
#     'SV001': {'missed': {'Accounting': 2}, 'registered_sessions': 3}, 'SV002': {'missed': {}, 'registered_sessions': 3}, 'SV003': {'missed': {'DSA': 1}, 'registered_sessions': 3}, 'SV004': {'missed': {}, 'registered_sessions': 3}, 'SV005': {'missed': {'Literature Analysis': 2}, 'registered_sessions': 4},
#     'SV006': {'missed': {}, 'registered_sessions': 3}, 'SV007': {'missed': {'Advanced Programming': 1}, 'registered_sessions': 3}, 'SV008': {'missed': {}, 'registered_sessions': 3}, 'SV009': {'missed': {'DSA': 2}, 'registered_sessions': 3}, 'SV010': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
#     'SV011': {'missed': {}, 'registered_sessions': 3}, 'SV012': {'missed': {'Auditing': 2}, 'registered_sessions': 3}, 'SV013': {'missed': {}, 'registered_sessions': 3}, 'SV014': {'missed': {'Advanced Programming': 1}, 'registered_sessions': 3}, 'SV015': {'missed': {'DSA': 3}, 'registered_sessions': 3},
#     'SV016': {'missed': {}, 'registered_sessions': 3}, 'SV017': {'missed': {'Machine Learning': 1}, 'registered_sessions': 3}, 'SV018': {'missed': {}, 'registered_sessions': 3}, 'SV019': {'missed': {'Auditing': 2}, 'registered_sessions': 3}, 'SV020': {'missed': {'Machine Learning': 1}, 'registered_sessions': 4},
#     'SV021': {'missed': {}, 'registered_sessions': 3}, 'SV022': {'missed': {'Human Resources': 2}, 'registered_sessions': 3}, 'SV023': {'missed': {}, 'registered_sessions': 3}, 'SV024': {'missed': {'Machine Learning': 1}, 'registered_sessions': 3}, 'SV025': {'missed': {'Literature Analysis': 3}, 'registered_sessions': 4},
#     'SV026': {'missed': {}, 'registered_sessions': 3}, 'SV027': {'missed': {'Computer Networks': 1}, 'registered_sessions': 3}, 'SV028': {'missed': {}, 'registered_sessions': 3}, 'SV029': {'missed': {'Human Resources': 2}, 'registered_sessions': 3}, 'SV030': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
#     'SV031': {'missed': {}, 'registered_sessions': 3}, 'SV032': {'missed': {'Investment': 2}, 'registered_sessions': 3}, 'SV033': {'missed': {}, 'registered_sessions': 3}, 'SV034': {'missed': {'Computer Networks': 1}, 'registered_sessions': 3}, 'SV035': {'missed': {'Literature Analysis': 3}, 'registered_sessions': 4},
#     'SV036': {'missed': {}, 'registered_sessions': 3}, 'SV037': {'missed': {'Advanced Programming': 1}, 'registered_sessions': 3}, 'SV038': {'missed': {}, 'registered_sessions': 3}, 'SV039': {'missed': {'DSA': 2}, 'registered_sessions': 3}, 'SV040': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
#     'SV041': {'missed': {}, 'registered_sessions': 3}, 'SV042': {'missed': {'Business Analytics': 2}, 'registered_sessions': 3}, 'SV043': {'missed': {}, 'registered_sessions': 3}, 'SV044': {'missed': {'DSA': 1}, 'registered_sessions': 3}, 'SV045': {'missed': {'Literature Analysis': 3}, 'registered_sessions': 4},
#     'SV046': {'missed': {}, 'registered_sessions': 3}, 'SV047': {'missed': {'Software Engineering': 1}, 'registered_sessions': 3}, 'SV048': {'missed': {}, 'registered_sessions': 3}, 'SV049': {'missed': {'DSA': 2}, 'registered_sessions': 3}, 'SV050': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
#     'SV051': {'missed': {}, 'registered_sessions': 2}, 'SV052': {'missed': {'Microeconomics': 2}, 'registered_sessions': 2}, 'SV053': {'missed': {}, 'registered_sessions': 2}, 'SV054': {'missed': {'DSA': 1}, 'registered_sessions': 2}, 'SV055': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
#     'SV056': {'missed': {}, 'registered_sessions': 2}, 'SV057': {'missed': {'Machine Learning': 1}, 'registered_sessions': 2}, 'SV058': {'missed': {}, 'registered_sessions': 2}, 'SV059': {'missed': {'Microeconomics': 2}, 'registered_sessions': 2}, 'SV060': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
#     'SV061': {'missed': {}, 'registered_sessions': 2}, 'SV062': {'missed': {'Macroeconomics': 2}, 'registered_sessions': 2}, 'SV063': {'missed': {}, 'registered_sessions': 2}, 'SV064': {'missed': {'Machine Learning': 1}, 'registered_sessions': 2}, 'SV065': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
#     'SV066': {'missed': {}, 'registered_sessions': 2}, 'SV067': {'missed': {'Macroeconomics': 1}, 'registered_sessions': 2}, 'SV068': {'missed': {}, 'registered_sessions': 2}, 'SV069': {'missed': {'Machine Learning': 2}, 'registered_sessions': 2}, 'SV070': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
#     'SV071': {'missed': {}, 'registered_sessions': 2}, 'SV072': {'missed': {'Sociology': 2}, 'registered_sessions': 2}, 'SV073': {'missed': {}, 'registered_sessions': 2}, 'SV074': {'missed': {'Machine Learning': 1}, 'registered_sessions': 2}, 'SV075': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
#     'SV076': {'missed': {}, 'registered_sessions': 2}, 'SV077': {'missed': {'Sociology': 1}, 'registered_sessions': 2}, 'SV078': {'missed': {}, 'registered_sessions': 2}, 'SV079': {'missed': {'Computer Networks': 2}, 'registered_sessions': 2}, 'SV080': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
#     'SV081': {'missed': {}, 'registered_sessions': 2}, 'SV082': {'missed': {'Psychology': 2}, 'registered_sessions': 2}, 'SV083': {'missed': {}, 'registered_sessions': 2}, 'SV084': {'missed': {'Computer Networks': 1}, 'registered_sessions': 2}, 'SV085': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
#     'SV086': {'missed': {}, 'registered_sessions': 2}, 'SV087': {'missed': {'Psychology': 1}, 'registered_sessions': 2}, 'SV088': {'missed': {}, 'registered_sessions': 2}, 'SV089': {'missed': {'Computer Networks': 2}, 'registered_sessions': 2}, 'SV090': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
#     'SV091': {'missed': {}, 'registered_sessions': 2}, 'SV092': {'missed': {'English C1': 2}, 'registered_sessions': 2}, 'SV093': {'missed': {}, 'registered_sessions': 2}, 'SV094': {'missed': {'Software Engineering': 1}, 'registered_sessions': 2}, 'SV095': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
#     'SV096': {'missed': {}, 'registered_sessions': 2}, 'SV097': {'missed': {'English C1': 1}, 'registered_sessions': 2}, 'SV098': {'missed': {}, 'registered_sessions': 2}, 'SV099': {'missed': {'Software Engineering': 2}, 'registered_sessions': 2}, 'SV100': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
#     'SV101': {'missed': {}, 'registered_sessions': 2}, 'SV102': {'missed': {'English C1': 2}, 'registered_sessions': 2}, 'SV103': {'missed': {}, 'registered_sessions': 2}, 'SV104': {'missed': {'Software Engineering': 1}, 'registered_sessions': 2}, 'SV105': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
#     'SV106': {'missed': {}, 'registered_sessions': 2}, 'SV107': {'missed': {'English C1': 1}, 'registered_sessions': 2}, 'SV108': {'missed': {}, 'registered_sessions': 2}, 'SV109': {'missed': {'Software Engineering': 2}, 'registered_sessions': 2}, 'SV110': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
#     'SV111': {'missed': {}, 'registered_sessions': 3}, 'SV112': {'missed': {'Japanese N3': 2}, 'registered_sessions': 3}, 'SV113': {'missed': {}, 'registered_sessions': 3}, 'SV114': {'missed': {'Accounting': 1}, 'registered_sessions': 3}, 'SV115': {'missed': {'Intl Relations': 3}, 'registered_sessions': 4},
#     'SV116': {'missed': {}, 'registered_sessions': 3}, 'SV117': {'missed': {'Japanese N3': 1}, 'registered_sessions': 3}, 'SV118': {'missed': {}, 'registered_sessions': 3}, 'SV119': {'missed': {'Auditing': 2}, 'registered_sessions': 3}, 'SV120': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
#     'SV121': {'missed': {}, 'registered_sessions': 3}, 'SV122': {'missed': {'Korean Topik 2': 2}, 'registered_sessions': 3}, 'SV123': {'missed': {}, 'registered_sessions': 3}, 'SV124': {'missed': {'Human Resources': 1}, 'registered_sessions': 3}, 'SV125': {'missed': {'Intl Relations': 3}, 'registered_sessions': 4},
#     'SV126': {'missed': {}, 'registered_sessions': 3}, 'SV127': {'missed': {'Korean Topik 2': 1}, 'registered_sessions': 3}, 'SV128': {'missed': {}, 'registered_sessions': 3}, 'SV129': {'missed': {'Investment': 2}, 'registered_sessions': 3}, 'SV130': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
#     'SV131': {'missed': {}, 'registered_sessions': 3}, 'SV132': {'missed': {'Sociology': 2}, 'registered_sessions': 3}, 'SV133': {'missed': {}, 'registered_sessions': 3}, 'SV134': {'missed': {'Business Analytics': 1}, 'registered_sessions': 3}, 'SV135': {'missed': {'Intl Relations': 3}, 'registered_sessions': 4},
#     'SV136': {'missed': {}, 'registered_sessions': 2}, 'SV137': {'missed': {'Sociology': 1}, 'registered_sessions': 2}, 'SV138': {'missed': {}, 'registered_sessions': 2}, 'SV139': {'missed': {'Korean Topik 2': 2}, 'registered_sessions': 2}, 'SV140': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
#     'SV141': {'missed': {}, 'registered_sessions': 2}, 'SV142': {'missed': {'Psychology': 2}, 'registered_sessions': 2}, 'SV143': {'missed': {}, 'registered_sessions': 2}, 'SV144': {'missed': {'Literature Analysis': 1}, 'registered_sessions': 2}, 'SV145': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
#     'SV146': {'missed': {}, 'registered_sessions': 2}, 'SV147': {'missed': {'Psychology': 1}, 'registered_sessions': 2}, 'SV148': {'missed': {}, 'registered_sessions': 2}, 'SV149': {'missed': {'Literature Analysis': 2}, 'registered_sessions': 2}, 'SV150': {'missed': {'Project Management': 1}, 'registered_sessions': 2},
# }

""" End Test Data lớn """

""" Test Data nhỏ """
MIN_STUDENT_PER_CLASS = 3
STUDENT_BASE_WEIGHT = 1000
MISSED_SESSION_WEIGHT = 5
REGISTERED_SESSIONS_WEIGHT = 20
CLASS_OPEN_PENALTY = -2500
CAPACITY_PENALTY_FACTOR = 1
sessions = {
    "SS1": {
        "subject": "Math",
        "facility": "NEU",
        "slots": [{"begin": 8, "end": 10}, {"begin": 14, "end": 16}],
        "registers": ["alice", "bob", "charlie", "david", "eva", "frank", "grace"]
    },
    "SS2": {
        "subject": "Physics",
        "facility": "NEU",
        "slots": [{"begin": 10, "end": 12}],
        "registers": ["alice", "bob", "charlie", "david", "eva"]
    },
    "SS3": {
        "subject": "English",
        "facility": "FTU",
        "slots": [{"begin": 9, "end": 11}, {"begin": 13, "end": 15}],
        "registers": ["alice", "charlie", "henry", "iris", "jack", "kate"]
    },
    "SS4": {
        "subject": "Chemistry",
        "facility": "FTU",
        "slots": [{"begin": 15, "end": 17}],
        "registers": ["bob", "david", "henry", "iris"]
    }
}

rooms = {
    "NEU_R1": {
        "busy": [{"begin": 12, "end": 13}],
        "facility": "NEU",
        "capacity": 4
    },
    "NEU_R2": {
        "busy": [{"begin": 7, "end": 8}, {"begin": 16, "end": 18}],
        "facility": "NEU",
        "capacity": 6
    },
    "FTU_R1": {
        "busy": [{"begin": 11, "end": 13}],
        "facility": "FTU",
        "capacity": 3
    },
    "FTU_R2": {
        "busy": [{"begin": 17, "end": 19}],
        "facility": "FTU",
        "capacity": 8
    }
}

students = {
    "alice": {"missed": {"Math": 3, "English": 1}, "registered_sessions": 3},
    "bob": {"missed": {"Physics": 2}, "registered_sessions": 3},
    "charlie": {"missed": {"Math": 1, "English": 2}, "registered_sessions": 3},
    "david": {"missed": {"Chemistry": 1}, "registered_sessions": 3},
    "eva": {"missed": {}, "registered_sessions": 2},
    "frank": {"missed": {"Math": 4}, "registered_sessions": 1},
    "grace": {"missed": {"Math": 2}, "registered_sessions": 1},
    "henry": {"missed": {"English": 3, "Chemistry": 2}, "registered_sessions": 2},
    "iris": {"missed": {"English": 1, "Chemistry": 3}, "registered_sessions": 2},
    "jack": {"missed": {"English": 2}, "registered_sessions": 1},
    "kate": {"missed": {}, "registered_sessions": 1}
}
""" End Test Data nhỏ """

# =============================================================================
# Các hàm phụ trợ (HELPER FUNCTIONS)
# =============================================================================

def slots_conflict(slot_a, slot_b):
    """Kiểm tra 2 slot có trùng nhau không."""
    return not (slot_a["end"] <= slot_b["begin"] or slot_b["end"] <= slot_a["begin"])


def room_busy_conflict(room_busy_slots, class_slot):
    """Kiểm tra slot có trùng với lịch bận của phòng."""
    return any(slots_conflict(busy_slot, class_slot) for busy_slot in room_busy_slots)


# =============================================================================
# Các hàm chính của mô hình (CORE MODEL FUNCTIONS)
# =============================================================================

def create_decision_variables(solver, sessions, rooms, classes_by_session):
    """Tạo tất cả các biến quyết định cho bài toán."""
    print("\n=== 🔧 Tạo biến quyết định ===")
    is_class_open = {
        class_id: solver.BoolVar(f"is_class_open_{class_id}")
        for session_id in sessions
        for class_id in classes_by_session[session_id]
    }

    is_room_assigned_to_class = {
        (class_id, room_id): solver.BoolVar(f"is_room_{room_id}_for_{class_id}")
        for session_id, session in sessions.items()
        for class_id in classes_by_session[session_id]
        for room_id, room in rooms.items()
        if room["facility"] == session["facility"]
    }

    is_student_assigned_to_class = {
        (student_id, class_id): solver.BoolVar(f"is_student_{student_id}_in_{class_id}")
        for session_id, session in sessions.items()
        for class_id in classes_by_session[session_id]
        for student_id in session["registers"]
    }

    print(f"Số biến is_class_open: {len(is_class_open)}")
    print(f"Số biến is_room_assigned_to_class: {len(is_room_assigned_to_class)}")
    print(f"Số biến is_student_assigned_to_class: {len(is_student_assigned_to_class)}")

    return is_class_open, is_room_assigned_to_class, is_student_assigned_to_class


def add_constraints(solver, data, variables):
    """Thêm tất cả các ràng buộc vào solver."""
    print("\n=== 📏 Thêm ràng buộc ===")
    sessions, rooms, students, classes_by_session = data
    is_class_open, is_room_assigned, is_student_assigned = variables

    # [1] Nếu lớp mở => phải chọn đúng 1 phòng
    for class_list in classes_by_session.values():
        for class_id in class_list:
            possible_rooms = [is_room_assigned[class_id, r_id] for r_id in rooms if (class_id, r_id) in is_room_assigned]
            solver.Add(solver.Sum(possible_rooms) == is_class_open[class_id])

    # [2] Không trùng lịch bận phòng
    for session_id, session in sessions.items():
        for class_id in classes_by_session[session_id]:
            for room_id, room in rooms.items():
                if (class_id, room_id) in is_room_assigned:
                    for slot in session["slots"]:
                        if room_busy_conflict(room["busy"], slot):
                            solver.Add(is_room_assigned[class_id, room_id] == 0)

    # [3] Sức chứa phòng
    for session_id, session in sessions.items():
        students_in_session = session["registers"]
        big_m = len(students_in_session)
        for class_id in classes_by_session[session_id]:
            num_students_in_class = solver.Sum(is_student_assigned[s_id, class_id] for s_id in students_in_session)
            for room_id, room in rooms.items():
                if (class_id, room_id) in is_room_assigned:
                    solver.Add(num_students_in_class <= room["capacity"] + big_m * (1 - is_room_assigned[class_id, room_id]))

    # [4] Số học viên tối thiểu để mở lớp
    for session_id, session in sessions.items():
        for class_id in classes_by_session[session_id]:
            num_students = solver.Sum(is_student_assigned[s_id, class_id] for s_id in session["registers"])
            solver.Add(num_students >= MIN_STUDENT_PER_CLASS * is_class_open[class_id])

    # [5] Học viên chỉ học 1 lớp của mỗi session
    for session_id, session in sessions.items():
        for student_id in session["registers"]:
            solver.Add(solver.Sum(is_student_assigned[student_id, c_id] for c_id in classes_by_session[session_id]) <= 1)

    # [6] Nếu SV được xếp thì lớp phải mở
    for student_id, class_id in is_student_assigned:
        solver.Add(is_student_assigned[student_id, class_id] <= is_class_open[class_id])

    # [7] Không cho SV học trùng giờ khác môn
    sessions_by_student = {s_id: [ss_id for ss_id, ss in sessions.items() if s_id in ss["registers"]] for s_id in students}
    for student_id, session_list in sessions_by_student.items():
        for i in range(len(session_list)):
            for j in range(i + 1, len(session_list)):
                s_a_id, s_b_id = session_list[i], session_list[j]
                session_a, session_b = sessions[s_a_id], sessions[s_b_id]
                if session_a["subject"] != session_b["subject"] and any(slots_conflict(s_a, s_b) for s_a in session_a["slots"] for s_b in session_b["slots"]):
                    for class_a in classes_by_session[s_a_id]:
                        for class_b in classes_by_session[s_b_id]:
                            solver.Add(is_student_assigned[student_id, class_a] + is_student_assigned[student_id, class_b] <= 1)

    # [8] Mỗi phòng chỉ có 1 lớp trong cùng thời gian
    for room_id in rooms:
        class_slots = []
        for session_id, session in sessions.items():
            if (list(classes_by_session[session_id])[0], room_id) in is_room_assigned:
                for class_id in classes_by_session[session_id]:
                    class_slots.append((class_id, session["slots"]))
        for i in range(len(class_slots)):
            for j in range(i + 1, len(class_slots)):
                class_a, slots_a = class_slots[i]
                class_b, slots_b = class_slots[j]
                if any(slots_conflict(sa, sb) for sa in slots_a for sb in slots_b):
                    solver.Add(is_room_assigned[class_a, room_id] + is_room_assigned[class_b, room_id] <= 1)


def define_objective(solver, data, variables):
    """Xây dựng hàm mục tiêu."""
    print("\n=== 🎯 Xây dựng hàm mục tiêu ===")
    sessions, rooms, students, classes_by_session = data
    is_class_open, is_room_assigned, is_student_assigned = variables

    objective = solver.Objective()

    # Thêm điểm thưởng/phạt vào hàm mục tiêu
    for class_id, var in is_class_open.items():
        objective.SetCoefficient(var, CLASS_OPEN_PENALTY)

    for (class_id, room_id), var in is_room_assigned.items():
        penalty = -1 * CAPACITY_PENALTY_FACTOR * rooms[room_id]['capacity']
        objective.SetCoefficient(var, penalty)

    for (student_id, class_id), var in is_student_assigned.items():
        session_id = class_id.split('_C')[0]
        session = sessions[session_id]
        student_info = students[student_id]
        weight = STUDENT_BASE_WEIGHT
        weight += REGISTERED_SESSIONS_WEIGHT * (1 / student_info["registered_sessions"])
        weight += MISSED_SESSION_WEIGHT * student_info["missed"].get(session["subject"], 0)
        objective.SetCoefficient(var, weight)

    objective.SetMaximization()


def solve_and_print_results(solver, data, variables, students):
    """Giải bài toán và in kết quả."""
    print("\n=== 🚀 Bắt đầu giải bài toán ===")
    sessions, rooms, _, classes_by_session = data
    is_class_open, is_room_assigned, is_student_assigned = variables

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("\n✅ Lời giải tối ưu đã được tìm thấy.")
        print_statistics(sessions, students, classes_by_session, is_class_open, is_student_assigned)
        print("\n Danh sách các lớp có thể mở: \n")
        for session_id, session in sessions.items():
            for class_id in classes_by_session[session_id]:
                if is_class_open[class_id].solution_value() > 0.5:
                    room_id = next(r_id for (c_id, r_id), var in is_room_assigned.items() if c_id == class_id and var.solution_value() > 0.5)
                    assigned_students = [s_id for (s_id, c_id), var in is_student_assigned.items() if c_id == class_id and var.solution_value() > 0.5]

                    print(f"📌 Lớp {class_id} | Phòng: {room_id} "
                          f"| Sĩ số: {len(assigned_students)}/{rooms[room_id]['capacity']} "
                          f"| Học viên: {assigned_students}")
    else:
        print("\n❌ Không tìm thấy lời giải tối ưu.")
    print("\n=== Các tham số điều chỉnh bài toán ===\n")
    print(f"Số học viên tối thiểu để mở một lớp (MIN_STUDENT_PER_CLASS): {MIN_STUDENT_PER_CLASS}")
    print(f"Trọng số cơ bản khi xếp được một sinh viên vào lớp (STUDENT_BASE_WEIGHT): {STUDENT_BASE_WEIGHT}")
    print(f"Điểm cộng cho mỗi buổi học sinh viên đã lỡ (MISSED_SESSION_WEIGHT): {MISSED_SESSION_WEIGHT}")
    print(f"Điểm cộng dựa trên số session đã đăng ký (REGISTERED_SESSIONS_WEIGHT): {REGISTERED_SESSIONS_WEIGHT}")
    print(f"Điểm phạt khi mở một lớp mới (CLASS_OPEN_PENALTY): {CLASS_OPEN_PENALTY}")
    print(f"Hệ số phạt cho việc sử dụng phòng lớn (CAPACITY_PENALTY_FACTOR): {CAPACITY_PENALTY_FACTOR}")
    print("\n=== 🏁 Kết thúc giải bài toán ===")

def print_statistics(sessions, students, classes_by_session, is_class_open, is_student_assigned):
    """In thông tin thống kê từ kết quả giải."""
    print("\n=== 📊 Thống kê kết quả ===")

    # Sessions khả thi (tất cả sessions trong dữ liệu đầu vào)
    num_possible_sessions = len(sessions)
    print(f"- Sessions khả thi: {num_possible_sessions}")

    # Số Sinh viên được phục vụ
    assigned_students_set = set()
    for (student_id, class_id), var in is_student_assigned.items():
        if var.solution_value() > 0.5:
            assigned_students_set.add(student_id)
    num_assigned_students = len(assigned_students_set)
    print(f"- Số Sinh viên được phục vụ: {num_assigned_students}")

    # Lớp được mở
    opened_classes = [class_id for class_id, var in is_class_open.items() if var.solution_value() > 0.5]
    num_opened_classes = len(opened_classes)
    print(f"- Lớp được mở: {num_opened_classes}")

    # Các sinh viên không được phục vụ
    all_students = set(students.keys())
    unassigned_students = all_students - assigned_students_set
    print(f"- Các sinh viên không được phục vụ: {list(unassigned_students)}")


def run_scheduler(sessions, rooms, students):
    """Hàm chính điều phối toàn bộ quá trình."""
    print("=== 🔍 Khởi tạo solver ===")
    solver = pywraplp.Solver.CreateSolver("SCIP")

    # Tạo danh sách các lớp học tiềm năng
    print("\n=== 🏫 Tạo danh sách lớp cho mỗi session ===")
    classes_by_session = {}
    for session_id, session in sessions.items():
        max_c = math.floor(len(session["registers"]) / MIN_STUDENT_PER_CLASS) or 1
        classes_by_session[session_id] = [f"{session_id}_C{i}" for i in range(max_c)]
        print(f"Session {session_id} -> Lớp: {classes_by_session[session_id]}")

    # Đóng gói dữ liệu để truyền cho các hàm
    data = (sessions, rooms, students, classes_by_session)

    # 1. Tạo biến quyết định
    variables = create_decision_variables(solver, sessions, rooms, classes_by_session) # Corrected arguments

    # 2. Thêm các ràng buộc
    add_constraints(solver, data, variables)

    # 3. Xây dựng hàm mục tiêu
    define_objective(solver, data, variables)

    # 4. Giải và in kết quả
    solve_and_print_results(solver, data, variables, students)


def main():
    # sessions = {
    #     # HUST - Tech
    #     "S01PROG": {"subject": "Advanced Programming", "facility": "HUST", "slots": [{"begin": 8, "end": 11}], "registers": ['SV001', 'SV002', 'SV003', 'SV004', 'SV005', 'SV006', 'SV007', 'SV008', 'SV009', 'SV010', 'SV011', 'SV012', 'SV013', 'SV014', 'SV015', 'SV016', 'SV017', 'SV018', 'SV019', 'SV020', 'SV021', 'SV022', 'SV023', 'SV024', 'SV025', 'SV026', 'SV027', 'SV028', 'SV029', 'SV030', 'SV031', 'SV032', 'SV033', 'SV034', 'SV035', 'SV036', 'SV037']},
    #     "S02DSA": {"subject": "DSA", "facility": "HUST", "slots": [{"begin": 13, "end": 16}], "registers": ['SV001', 'SV002', 'SV003', 'SV004', 'SV005', 'SV006', 'SV007', 'SV008', 'SV009', 'SV010', 'SV011', 'SV012', 'SV013', 'SV014', 'SV015', 'SV038', 'SV039', 'SV040', 'SV041', 'SV042', 'SV043', 'SV044', 'SV045', 'SV046', 'SV047', 'SV048', 'SV049', 'SV050', 'SV051', 'SV052', 'SV053', 'SV054', 'SV055', 'SV056']},
    #     "S03ML": {"subject": "Machine Learning", "facility": "HUST", "slots": [{"begin": 25, "end": 28}], "registers": ['SV016', 'SV017', 'SV018', 'SV019', 'SV020', 'SV021', 'SV022', 'SV023', 'SV024', 'SV025', 'SV057', 'SV058', 'SV059', 'SV060', 'SV061', 'SV062', 'SV063', 'SV064', 'SV065', 'SV066', 'SV067', 'SV068', 'SV069', 'SV070', 'SV071', 'SV072', 'SV073', 'SV074', 'SV075']},
    #     "S04CN": {"subject": "Computer Networks", "facility": "HUST", "slots": [{"begin": 28, "end": 31}], "registers": ['SV026', 'SV027', 'SV028', 'SV029', 'SV030', 'SV031', 'SV032', 'SV033', 'SV034', 'SV035', 'SV036', 'SV037', 'SV076', 'SV077', 'SV078', 'SV079', 'SV080', 'SV081', 'SV082', 'SV083', 'SV084', 'SV085', 'SV086', 'SV087', 'SV088', 'SV089', 'SV090', 'SV091', 'SV092', 'SV093']},
    #     "S05SE": {"subject": "Software Engineering", "facility": "HUST", "slots": [{"begin": 33, "end": 36}], "registers": ['SV038', 'SV039', 'SV040', 'SV041', 'SV042', 'SV043', 'SV044', 'SV045', 'SV046', 'SV047', 'SV048', 'SV049', 'SV050', 'SV094', 'SV095', 'SV096', 'SV097', 'SV098', 'SV099', 'SV100', 'SV101', 'SV102', 'SV103', 'SV104', 'SV105', 'SV106', 'SV107', 'SV108', 'SV109', 'SV110']},

    #     # VNU - Economics & Social Sciences
    #     "S06MICRO": {"subject": "Microeconomics", "facility": "VNU", "slots": [{"begin": 50, "end": 53}], "registers": ['SV051', 'SV052', 'SV053', 'SV054', 'SV055', 'SV056', 'SV057', 'SV058', 'SV059', 'SV060', 'SV111', 'SV112', 'SV113', 'SV114', 'SV115', 'SV116', 'SV117', 'SV118', 'SV119', 'SV120']},
    #     "S07MACRO": {"subject": "Macroeconomics", "facility": "VNU", "slots": [{"begin": 53, "end": 56}], "registers": ['SV061', 'SV062', 'SV063', 'SV064', 'SV065', 'SV066', 'SV067', 'SV068', 'SV069', 'SV070', 'SV121', 'SV122', 'SV123', 'SV124', 'SV125', 'SV126', 'SV127', 'SV128', 'SV129', 'SV130']},
    #     "S08SOC": {"subject": "Sociology", "facility": "VNU", "slots": [{"begin": 58, "end": 61}], "registers": ['SV071', 'SV072', 'SV073', 'SV074', 'SV075', 'SV076', 'SV077', 'SV078', 'SV079', 'SV080', 'SV131', 'SV132', 'SV133', 'SV134', 'SV135', 'SV136', 'SV137', 'SV138', 'SV139', 'SV140']},
    #     "S09PSY": {"subject": "Psychology", "facility": "VNU", "slots": [{"begin": 61, "end": 64}], "registers": ['SV081', 'SV082', 'SV083', 'SV084', 'SV085', 'SV086', 'SV087', 'SV088', 'SV089', 'SV090', 'SV141', 'SV142', 'SV143', 'SV144', 'SV145', 'SV146', 'SV147', 'SV148', 'SV149', 'SV150']},
    #     "S10PM": {"subject": "Project Management", "facility": "VNU", "slots": [{"begin": 65, "end": 68}], "registers": ['SV001', 'SV010', 'SV020', 'SV030', 'SV040', 'SV050', 'SV060', 'SV070', 'SV080', 'SV090', 'SV100', 'SV110', 'SV120', 'SV130', 'SV140', 'SV150']},

    #     # HANU - Languages & Culture
    #     "S11ENGC1": {"subject": "English C1", "facility": "HANU", "slots": [{"begin": 72, "end": 75}], "registers": ['SV091', 'SV092', 'SV093', 'SV094', 'SV095', 'SV096', 'SV097', 'SV098', 'SV099', 'SV100', 'SV101', 'SV102', 'SV103', 'SV104', 'SV105', 'SV106', 'SV107', 'SV108', 'SV109', 'SV110']},
    #     "S12JAPN3": {"subject": "Japanese N3", "facility": "HANU", "slots": [{"begin": 75, "end": 78}], "registers": ['SV111', 'SV112', 'SV113', 'SV114', 'SV115', 'SV116', 'SV117', 'SV118', 'SV119', 'SV120', 'SV121', 'SV122', 'SV123', 'SV124', 'SV125']},
    #     "S13KORT2": {"subject": "Korean Topik 2", "facility": "HANU", "slots": [{"begin": 80, "end": 83}], "registers": ['SV126', 'SV127', 'SV128', 'SV129', 'SV130', 'SV131', 'SV132', 'SV133', 'SV134', 'SV135', 'SV136', 'SV137', 'SV138', 'SV139', 'SV140']},
    #     "S14LIT": {"subject": "Literature Analysis", "facility": "HANU", "slots": [{"begin": 83, "end": 86}], "registers": ['SV141', 'SV142', 'SV143', 'SV144', 'SV145', 'SV146', 'SV147', 'SV148', 'SV149', 'SV150', 'SV005', 'SV015', 'SV025', 'SV035', 'SV045']},
    #     "S15IRS": {"subject": "Intl Relations", "facility": "HANU", "slots": [{"begin": 86, "end": 89}], "registers": ['SV055', 'SV065', 'SV075', 'SV085', 'SV095', 'SV105', 'SV115', 'SV125', 'SV135', 'SV145']},

    #     # NEU - Business & Finance
    #     "S16ACC": {"subject": "Accounting", "facility": "NEU", "slots": [{"begin": 96, "end": 99}], "registers": ['SV001', 'SV002', 'SV003', 'SV004', 'SV005', 'SV006', 'SV007', 'SV008', 'SV009', 'SV010', 'SV111', 'SV112', 'SV113', 'SV114', 'SV115']},
    #     "S17AUD": {"subject": "Auditing", "facility": "NEU", "slots": [{"begin": 99, "end": 102}], "registers": ['SV011', 'SV012', 'SV013', 'SV014', 'SV015', 'SV016', 'SV017', 'SV018', 'SV019', 'SV020', 'SV116', 'SV117', 'SV118', 'SV119', 'SV120']},
    #     "S18HRM": {"subject": "Human Resources", "facility": "NEU", "slots": [{"begin": 104, "end": 107}], "registers": ['SV021', 'SV022', 'SV023', 'SV024', 'SV025', 'SV026', 'SV027', 'SV028', 'SV029', 'SV030', 'SV121', 'SV122', 'SV123', 'SV124', 'SV125']},
    #     "S19INV": {"subject": "Investment", "facility": "NEU", "slots": [{"begin": 107, "end": 110}], "registers": ['SV031', 'SV032', 'SV033', 'SV034', 'SV035', 'SV036', 'SV037', 'SV038', 'SV039', 'SV040', 'SV126', 'SV127', 'SV128', 'SV129', 'SV130']},
    #     "S20BANA": {"subject": "Business Analytics", "facility": "NEU", "slots": [{"begin": 112, "end": 115}], "registers": ['SV041', 'SV042', 'SV043', 'SV044', 'SV045', 'SV046', 'SV047', 'SV048', 'SV049', 'SV050', 'SV131', 'SV132', 'SV133', 'SV134', 'SV135']}
    # }

    # rooms = {
    #     # HUST Rooms
    #     "HUST_D8_101": {"busy": [{"begin": 12, "end": 13}], "facility": "HUST", "capacity": 30},
    #     "HUST_D8_202": {"busy": [{"begin": 29, "end": 30}], "facility": "HUST", "capacity": 40},
    #     "HUST_TC_301": {"busy": [], "facility": "HUST", "capacity": 50},
    #     "HUST_TC_405": {"busy": [{"begin": 17, "end": 19}], "facility": "HUST", "capacity": 60},
    #     "HUST_D6_501": {"busy": [{"begin": 35, "end": 38}], "facility": "HUST", "capacity": 80},
    #     "HUST_D9_303": {"busy": [{"begin": 8, "end": 10}], "facility": "HUST", "capacity": 100},
    #     "HUST_D5_101": {"busy": [{"begin": 15, "end": 17}], "facility": "HUST", "capacity": 120},

    #     # VNU Rooms
    #     "VNU_E1_101": {"busy": [{"begin": 56, "end": 57}], "facility": "VNU", "capacity": 30},
    #     "VNU_E2_202": {"busy": [], "facility": "VNU", "capacity": 40},
    #     "VNU_G2_303": {"busy": [{"begin": 12, "end": 13}, {"begin": 56, "end": 57}], "facility": "VNU", "capacity": 50},
    #     "VNU_H1_404": {"busy": [{"begin": 62, "end": 65}], "facility": "VNU", "capacity": 60},
    #     "VNU_H2_505": {"busy": [{"begin": 50, "end": 52}], "facility": "VNU", "capacity": 80},
    #     "VNU_I1_201": {"busy": [{"begin": 68, "end": 70}], "facility": "VNU", "capacity": 100},
    #     "VNU_I2_302": {"busy": [], "facility": "VNU", "capacity": 120},

    #     # HANU Rooms
    #     "HANU_A1_101": {"busy": [{"begin": 78, "end": 79}], "facility": "HANU", "capacity": 30},
    #     "HANU_A2_202": {"busy": [{"begin": 102, "end": 103}], "facility": "HANU", "capacity": 40},
    #     "HANU_B2_303": {"busy": [], "facility": "HANU", "capacity": 50},
    #     "HANU_C5_404": {"busy": [{"begin": 83, "end": 85}], "facility": "HANU", "capacity": 60},
    #     "HANU_D3_505": {"busy": [{"begin": 73, "end": 74}], "facility": "HANU", "capacity": 80},

    #     # NEU Rooms
    #     "NEU_A1_101": {"busy": [], "facility": "NEU", "capacity": 30},
    #     "NEU_A1_202": {"busy": [{"begin": 102, "end": 104}], "facility": "NEU", "capacity": 40},
    #     "NEU_B2_303": {"busy": [{"begin": 98, "end": 99}], "facility": "NEU", "capacity": 50},
    #     "NEU_C1_404": {"busy": [], "facility": "NEU", "capacity": 60},
    #     "NEU_D2_505": {"busy": [{"begin": 110, "end": 112}], "facility": "NEU", "capacity": 80},
    #     "NEU_E1_201": {"busy": [{"begin": 96, "end": 98}], "facility": "NEU", "capacity": 100},
    #     "NEU_F2_302": {"busy": [], "facility": "NEU", "capacity": 120}
    # }

    # students = {
    #     'SV001': {'missed': {'Accounting': 2}, 'registered_sessions': 3}, 'SV002': {'missed': {}, 'registered_sessions': 3}, 'SV003': {'missed': {'DSA': 1}, 'registered_sessions': 3}, 'SV004': {'missed': {}, 'registered_sessions': 3}, 'SV005': {'missed': {'Literature Analysis': 2}, 'registered_sessions': 4},
    #     'SV006': {'missed': {}, 'registered_sessions': 3}, 'SV007': {'missed': {'Advanced Programming': 1}, 'registered_sessions': 3}, 'SV008': {'missed': {}, 'registered_sessions': 3}, 'SV009': {'missed': {'DSA': 2}, 'registered_sessions': 3}, 'SV010': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
    #     'SV011': {'missed': {}, 'registered_sessions': 3}, 'SV012': {'missed': {'Auditing': 2}, 'registered_sessions': 3}, 'SV013': {'missed': {}, 'registered_sessions': 3}, 'SV014': {'missed': {'Advanced Programming': 1}, 'registered_sessions': 3}, 'SV015': {'missed': {'DSA': 3}, 'registered_sessions': 3},
    #     'SV016': {'missed': {}, 'registered_sessions': 3}, 'SV017': {'missed': {'Machine Learning': 1}, 'registered_sessions': 3}, 'SV018': {'missed': {}, 'registered_sessions': 3}, 'SV019': {'missed': {'Auditing': 2}, 'registered_sessions': 3}, 'SV020': {'missed': {'Machine Learning': 1}, 'registered_sessions': 4},
    #     'SV021': {'missed': {}, 'registered_sessions': 3}, 'SV022': {'missed': {'Human Resources': 2}, 'registered_sessions': 3}, 'SV023': {'missed': {}, 'registered_sessions': 3}, 'SV024': {'missed': {'Machine Learning': 1}, 'registered_sessions': 3}, 'SV025': {'missed': {'Literature Analysis': 3}, 'registered_sessions': 4},
    #     'SV026': {'missed': {}, 'registered_sessions': 3}, 'SV027': {'missed': {'Computer Networks': 1}, 'registered_sessions': 3}, 'SV028': {'missed': {}, 'registered_sessions': 3}, 'SV029': {'missed': {'Human Resources': 2}, 'registered_sessions': 3}, 'SV030': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
    #     'SV031': {'missed': {}, 'registered_sessions': 3}, 'SV032': {'missed': {'Investment': 2}, 'registered_sessions': 3}, 'SV033': {'missed': {}, 'registered_sessions': 3}, 'SV034': {'missed': {'Computer Networks': 1}, 'registered_sessions': 3}, 'SV035': {'missed': {'Literature Analysis': 3}, 'registered_sessions': 4},
    #     'SV036': {'missed': {}, 'registered_sessions': 3}, 'SV037': {'missed': {'Advanced Programming': 1}, 'registered_sessions': 3}, 'SV038': {'missed': {}, 'registered_sessions': 3}, 'SV039': {'missed': {'DSA': 2}, 'registered_sessions': 3}, 'SV040': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
    #     'SV041': {'missed': {}, 'registered_sessions': 3}, 'SV042': {'missed': {'Business Analytics': 2}, 'registered_sessions': 3}, 'SV043': {'missed': {}, 'registered_sessions': 3}, 'SV044': {'missed': {'DSA': 1}, 'registered_sessions': 3}, 'SV045': {'missed': {'Literature Analysis': 3}, 'registered_sessions': 4},
    #     'SV046': {'missed': {}, 'registered_sessions': 3}, 'SV047': {'missed': {'Software Engineering': 1}, 'registered_sessions': 3}, 'SV048': {'missed': {}, 'registered_sessions': 3}, 'SV049': {'missed': {'DSA': 2}, 'registered_sessions': 3}, 'SV050': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
    #     'SV051': {'missed': {}, 'registered_sessions': 2}, 'SV052': {'missed': {'Microeconomics': 2}, 'registered_sessions': 2}, 'SV053': {'missed': {}, 'registered_sessions': 2}, 'SV054': {'missed': {'DSA': 1}, 'registered_sessions': 2}, 'SV055': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
    #     'SV056': {'missed': {}, 'registered_sessions': 2}, 'SV057': {'missed': {'Machine Learning': 1}, 'registered_sessions': 2}, 'SV058': {'missed': {}, 'registered_sessions': 2}, 'SV059': {'missed': {'Microeconomics': 2}, 'registered_sessions': 2}, 'SV060': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
    #     'SV061': {'missed': {}, 'registered_sessions': 2}, 'SV062': {'missed': {'Macroeconomics': 2}, 'registered_sessions': 2}, 'SV063': {'missed': {}, 'registered_sessions': 2}, 'SV064': {'missed': {'Machine Learning': 1}, 'registered_sessions': 2}, 'SV065': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
    #     'SV066': {'missed': {}, 'registered_sessions': 2}, 'SV067': {'missed': {'Macroeconomics': 1}, 'registered_sessions': 2}, 'SV068': {'missed': {}, 'registered_sessions': 2}, 'SV069': {'missed': {'Machine Learning': 2}, 'registered_sessions': 2}, 'SV070': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
    #     'SV071': {'missed': {}, 'registered_sessions': 2}, 'SV072': {'missed': {'Sociology': 2}, 'registered_sessions': 2}, 'SV073': {'missed': {}, 'registered_sessions': 2}, 'SV074': {'missed': {'Machine Learning': 1}, 'registered_sessions': 2}, 'SV075': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
    #     'SV076': {'missed': {}, 'registered_sessions': 2}, 'SV077': {'missed': {'Sociology': 1}, 'registered_sessions': 2}, 'SV078': {'missed': {}, 'registered_sessions': 2}, 'SV079': {'missed': {'Computer Networks': 2}, 'registered_sessions': 2}, 'SV080': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
    #     'SV081': {'missed': {}, 'registered_sessions': 2}, 'SV082': {'missed': {'Psychology': 2}, 'registered_sessions': 2}, 'SV083': {'missed': {}, 'registered_sessions': 2}, 'SV084': {'missed': {'Computer Networks': 1}, 'registered_sessions': 2}, 'SV085': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
    #     'SV086': {'missed': {}, 'registered_sessions': 2}, 'SV087': {'missed': {'Psychology': 1}, 'registered_sessions': 2}, 'SV088': {'missed': {}, 'registered_sessions': 2}, 'SV089': {'missed': {'Computer Networks': 2}, 'registered_sessions': 2}, 'SV090': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
    #     'SV091': {'missed': {}, 'registered_sessions': 2}, 'SV092': {'missed': {'English C1': 2}, 'registered_sessions': 2}, 'SV093': {'missed': {}, 'registered_sessions': 2}, 'SV094': {'missed': {'Software Engineering': 1}, 'registered_sessions': 2}, 'SV095': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
    #     'SV096': {'missed': {}, 'registered_sessions': 2}, 'SV097': {'missed': {'English C1': 1}, 'registered_sessions': 2}, 'SV098': {'missed': {}, 'registered_sessions': 2}, 'SV099': {'missed': {'Software Engineering': 2}, 'registered_sessions': 2}, 'SV100': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
    #     'SV101': {'missed': {}, 'registered_sessions': 2}, 'SV102': {'missed': {'English C1': 2}, 'registered_sessions': 2}, 'SV103': {'missed': {}, 'registered_sessions': 2}, 'SV104': {'missed': {'Software Engineering': 1}, 'registered_sessions': 2}, 'SV105': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
    #     'SV106': {'missed': {}, 'registered_sessions': 2}, 'SV107': {'missed': {'English C1': 1}, 'registered_sessions': 2}, 'SV108': {'missed': {}, 'registered_sessions': 2}, 'SV109': {'missed': {'Software Engineering': 2}, 'registered_sessions': 2}, 'SV110': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
    #     'SV111': {'missed': {}, 'registered_sessions': 3}, 'SV112': {'missed': {'Japanese N3': 2}, 'registered_sessions': 3}, 'SV113': {'missed': {}, 'registered_sessions': 3}, 'SV114': {'missed': {'Accounting': 1}, 'registered_sessions': 3}, 'SV115': {'missed': {'Intl Relations': 3}, 'registered_sessions': 4},
    #     'SV116': {'missed': {}, 'registered_sessions': 3}, 'SV117': {'missed': {'Japanese N3': 1}, 'registered_sessions': 3}, 'SV118': {'missed': {}, 'registered_sessions': 3}, 'SV119': {'missed': {'Auditing': 2}, 'registered_sessions': 3}, 'SV120': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
    #     'SV121': {'missed': {}, 'registered_sessions': 3}, 'SV122': {'missed': {'Korean Topik 2': 2}, 'registered_sessions': 3}, 'SV123': {'missed': {}, 'registered_sessions': 3}, 'SV124': {'missed': {'Human Resources': 1}, 'registered_sessions': 3}, 'SV125': {'missed': {'Intl Relations': 3}, 'registered_sessions': 4},
    #     'SV126': {'missed': {}, 'registered_sessions': 3}, 'SV127': {'missed': {'Korean Topik 2': 1}, 'registered_sessions': 3}, 'SV128': {'missed': {}, 'registered_sessions': 3}, 'SV129': {'missed': {'Investment': 2}, 'registered_sessions': 3}, 'SV130': {'missed': {'Project Management': 1}, 'registered_sessions': 4},
    #     'SV131': {'missed': {}, 'registered_sessions': 3}, 'SV132': {'missed': {'Sociology': 2}, 'registered_sessions': 3}, 'SV133': {'missed': {}, 'registered_sessions': 3}, 'SV134': {'missed': {'Business Analytics': 1}, 'registered_sessions': 3}, 'SV135': {'missed': {'Intl Relations': 3}, 'registered_sessions': 4},
    #     'SV136': {'missed': {}, 'registered_sessions': 2}, 'SV137': {'missed': {'Sociology': 1}, 'registered_sessions': 2}, 'SV138': {'missed': {}, 'registered_sessions': 2}, 'SV139': {'missed': {'Korean Topik 2': 2}, 'registered_sessions': 2}, 'SV140': {'missed': {'Project Management': 1}, 'registered_sessions': 3},
    #     'SV141': {'missed': {}, 'registered_sessions': 2}, 'SV142': {'missed': {'Psychology': 2}, 'registered_sessions': 2}, 'SV143': {'missed': {}, 'registered_sessions': 2}, 'SV144': {'missed': {'Literature Analysis': 1}, 'registered_sessions': 2}, 'SV145': {'missed': {'Intl Relations': 3}, 'registered_sessions': 3},
    #     'SV146': {'missed': {}, 'registered_sessions': 2}, 'SV147': {'missed': {'Psychology': 1}, 'registered_sessions': 2}, 'SV148': {'missed': {}, 'registered_sessions': 2}, 'SV149': {'missed': {'Literature Analysis': 2}, 'registered_sessions': 2}, 'SV150': {'missed': {'Project Management': 1}, 'registered_sessions': 2},
    # }

    run_scheduler(sessions, rooms, students)


if __name__ == "__main__":
    main()